# Time series

In [1]:
import pathlib

import panel as pn

pn.extension('tabulator', sizing_mode="stretch_width")
import hvplot.pandas
import holoviews as hv

# hv.extension('bokeh')
pn.extension('tabulator')
# pn.extension('bokeh')


import pandas as pd
from bokeh.io import show, output_notebook
from bokeh.plotting import figure, output_file, show
import bokeh.io


from bokeh.models.tools import Toolbar, HoverTool
from holoviews import dim, opts


In [2]:
print(hv.__version__, pd.__version__, pn.__version__, hvplot.__version__, bokeh.__version__)

1.14.9 1.2.4 0.12.1 0.8.0 2.3.3


In [3]:
# # Define paths.
# PATH_DATA = pathlib.Path(r'../data')

In [4]:
# db = pd.read_excel(PATH_DATA/'fusione5.xlsx', index_col='Unnamed: 0', dtype={'Year': int})
db = pd.read_excel('fusione5.xlsx', index_col='Unnamed: 0', dtype={'Year': int})

In [5]:
# Choose country
country = pn.widgets.Select(
#     name = 'Select Country',
    width = 200, height = 50,
    margin = [5,10],
    options = ['United Kingdom', 'Singapore', 'Occupied Palestinian Territory', 'Malaysia', 'Kazakhstan',
               'Turkey', 'Chile', 'Latvia', 'New Zealand', 'Switzerland', 'Serbia',
               'Argentina', 'Japan', 'Belarus', 'Hungary',  'Australia','Denmark',
               'Norway', 'Iceland', 'Poland', 'Bulgaria', 'France',
               'Belgium', 'Netherlands',  'Slovenia', 'Greece',  'Italy',
               'Ireland', 'Spain', 'Romania', 'Germany',
               'Finland', 'Lithuania', 'Portugal', 'Croatia', 'Sweden',  'Luxembourg', 'Austria']
)

# Choose country
country1 = pn.widgets.Select(
#     name = 'Select Country',
    width = 200, height = 50,
    margin = [5,10],
    options = ['United Kingdom', 'Singapore', 'Occupied Palestinian Territory', 'Malaysia', 'Kazakhstan',
               'Turkey', 'Chile', 'Latvia', 'New Zealand', 'Switzerland', 'Serbia',
               'Argentina', 'Japan', 'Belarus', 'Hungary',  'Australia','Denmark',
               'Norway', 'Iceland', 'Poland', 'Bulgaria', 'France',
               'Belgium', 'Netherlands',  'Slovenia', 'Greece',  'Italy',
               'Ireland', 'Spain', 'Romania', 'Germany',
               'Finland', 'Lithuania', 'Portugal', 'Croatia', 'Sweden',  'Luxembourg', 'Austria'],
    visible = False
)
not_euts = ['Singapore', 'Occupied Palestinian Territory', 'Malaysia', 'Kazakhstan',
        'Turkey', 'Chile', 'Latvia', 'New Zealand', 'Switzerland', 'Serbia',
        'Argentina', 'Japan', 'Belarus', 'Hungary',  'Australia']
euts = ['Denmark', 'Norway', 'Iceland', 'Poland', 'Bulgaria', 'France', 
        'Belgium', 'Netherlands',  'Slovenia', 'Greece',  'Italy',
        'Ireland', 'United Kingdom', 'Spain', 'Romania', 'Germany',
        'Finland', 'Lithuania', 'Portugal', 'Croatia', 'Sweden',  'Luxembourg', 'Austria']

country.jslink(country1, value='value', bidirectional=True)
pn.Row(country, country1)


Row(sizing_mode='stretch_width')
    [0] Select(height=50, margin=[5, 10], options=['United Kingdom', ...], sizing_mode='fixed', value='United Kingdom', width=200)
    [1] Select(height=50, margin=[5, 10], options=['United Kingdom', ...], sizing_mode='fixed', value='United Kingdom', visible=False, width=200)

In [6]:
# Define Palette  
# PALETTE = ["#d57883", "#04d9ff", "#003953", '#EE442F', '#F9F4EC', "#44749D", "#601A4A"]   #Start
# PALETTE = ["#d57883", "#7AC5CD", "#003953", '#EE442F', '#F9F4EC', '#44749D']#, "#darkgreen"] ## Second
PALETTE = ["#d57883", "#7AC5CD", "#003953", '#CAB8CB', '#F9F4EC', '#44749D']#, "#darkgreen"] ## Second
PALETTE_COLOR_BLINDNESS = ['#CCBE9F', '#EBE7E0', '#C6D4E1', '#CAB8CB', '#F9F4EC', '#44749D']#, '#601A4A']    #Start


# COLOR BLIND
# annual share #CCBE9F  marr chiaro
#kyoto 1 #EBE7E0  rosa
#kyoto2  #C6D4E1   lilla
# mean share #EE44SF   marrone
# banda #F9F4EC rosa chiarissimo/panna
# kyoto official #44749D   blu
# doha official #601A4A blu scuro

In [7]:
## Color widget for color blindness
select_mode_main = pn.widgets.RadioBoxGroup(name='Select Color', 
                                            options={'Standard': PALETTE[0], 'Color Blindness':  PALETTE_COLOR_BLINDNESS[0]}, 
                                            width = 200, height = 50,
                                            margin = [20,50],
                                            inline = True
                                            )

select_mode_kyotoI = pn.widgets.RadioBoxGroup(name='Select Vision', 
                                              options={'Standard': PALETTE[1], 'Color Blindness':  PALETTE_COLOR_BLINDNESS[1]}, 
                                              width = 200, height = 50,
                                              visible = False,
                                              margin = [5,10]
                                              )

select_mode_kyotoII = pn.widgets.RadioBoxGroup(name='Select Vision', 
                                               options={'Standard': PALETTE[2], 'Color Blindness':  PALETTE_COLOR_BLINDNESS[2]}, 
                                               width = 200, height = 50,
                                               visible = False,
                                               margin = [5,10]
                                              )

select_mode_mean = pn.widgets.RadioBoxGroup(name='Select Vision', 
                                              options={'Standard': PALETTE[3], 'Color Blindness':  PALETTE_COLOR_BLINDNESS[3]}, 
                                              width = 200, height = 50,
#                                               visible = False,
                                              margin = [5,10]
                                              )
## Combine color widget
select_mode_kyotoI.jslink(select_mode_main, value='value', bidirectional=True)
select_mode_kyotoII.jslink(select_mode_main, value='value', bidirectional=True)
select_mode_mean.jslink(select_mode_main, value='value', bidirectional=True)
# print(select_mode_main.value, select_mode_kyotoI.value, select_mode_kyotoII.value, select_mode_mean.value)
pn.Row(select_mode_main, select_mode_mean)


Row(sizing_mode='stretch_width')
    [0] RadioBoxGroup(height=50, inline=True, margin=[20, 50], name='Select Color', options={'Standard': '#d57883', ...}, sizing_mode='fixed', width=200)
    [1] RadioBoxGroup(height=50, margin=[5, 10], name='Select Vision', options={'Standard': '#CAB8CB', ...}, sizing_mode='fixed', value='#CAB8CB', width=200)

## Pipeline and interactive DataFrame

In [8]:
## Make DataFrame Pipeline Interactive
idc = db[['Entity', 'Code', 'Year', 'PercentageChangeProduction',
          'PercentageChange', 'Renewable electricity productions (TWh)',
          'Eco-friendly Production/Total Production Share(%)', 'Renewable electricity productions (TWh) Kyoto',
          'Eco-friendly Production/Total Production Share(%) Kyoto', 'Kyoto I', 'Kyoto II' 
         ]]

idf = idc.interactive()



# Define pipeline
ipipeline1 = (
      idf[
        (idf.Entity == country)
    ].round(4))
ipipeline1.head(1)


itable = ipipeline1.rename(columns={'PercentageChange': 
                                    'Relative Change Share (%)', 
                                    'PercentageChangeProduction':
                                    'Relative Change Production (%)',
                                    'Eco-friendly Production/Total Production Share(%)': 
                                    'Renewable Production/Total Production (%)'})[['Entity', 
                                                                                   'Code', 
                                                                                   'Year', 
                                                                                   'Relative Change Share (%)',
                                                                                   'Relative Change Production (%)',
                                                                                   'Renewable electricity productions (TWh)',
                                                                                   'Renewable Production/Total Production (%)']].pipe(pn.widgets.Tabulator, pagination='remote', 
                         page_size=10, sizing_mode = 'stretch_width', theme = 'bulma')
itable

## Share Time serie

### Combine mean plot and main plot

In [9]:
## Combine Plot and Pipeline
hover = HoverTool(tooltips=[("Year", "@{Year}"), 
                            ("Share", "@{Eco-friendly Production/Total Production Share(%)}%"),
                            ("Relative Change", "@{PercentageChange}%")
                           ],
                  mode = 'vline')

main_plot = ipipeline1.hvplot(x = 'Year',
                              y = 'Eco-friendly Production/Total Production Share(%)',
                              size = 'PercentageChange',
                              width=200, height=400,
                              alpha = 1, line_color= select_mode_main,
                              label = 'Annual Share'
                             ).opts(tools=['vline', 'box_zoom', 'reset'], toolbar='above')


kyotoI_plot = ipipeline1.hvplot(x = 'Kyoto I', 
                                y = 'Eco-friendly Production/Total Production Share(%) Kyoto',
                                width=200, height=400,
                                alpha = 1, line_color= select_mode_kyotoI,
                                label = 'Kyoto Protocol first part'
                                ).opts(tools=[""], toolbar=None)


kyotoII_plot = ipipeline1.hvplot(x = 'Kyoto II', 
                                 y = 'Eco-friendly Production/Total Production Share(%) Kyoto', 
                                 width=200, height=400, 
                                 label = 'Kyoto Protocol second part',
                                 alpha = 1, line_color= select_mode_kyotoII, 
                                 ).opts(tools=[""], 
                                        toolbar=None
                                       )
## Share mean band
mean_share = idc.groupby('Year').mean()[['Renewable electricity productions (TWh)', 'Eco-friendly Production/Total Production Share(%)']]
mean_share['Upper Share'] = mean_share['Eco-friendly Production/Total Production Share(%)'] + idc.groupby('Year').std()['Eco-friendly Production/Total Production Share(%)']
mean_share['Lower Share'] = mean_share['Eco-friendly Production/Total Production Share(%)'] - idc.groupby('Year').std()['Eco-friendly Production/Total Production Share(%)']

mean_s = mean_share.hvplot(x = 'Year',
                          y = 'Eco-friendly Production/Total Production Share(%)',
                          kind = 'line',
                          label = 'Mean Share',
                          alpha = 1, 
#                           line_color= 'red',
                          line_color= PALETTE[3],
                          ylim = (0,110),
                          xlim= (1990,2020),
                          )

band_s = mean_share.hvplot(x='Year', y='Upper Share', y2='Lower Share',
                          kind = 'area',
#                           color= 'red',
                           color = PALETTE[3],
                          alpha = 0.1
                         )

share_band = (mean_s * band_s.opts(tools = [], toolbar = None))


## Define interactive title
column_prod = pn.Column(pn.Row('## Renweable Energy Share: United Kingdom (In EU-ETS)'), 
                       pn.Row("Most of the countries with “very high” HDI (i.e. equal to or above 0.8) did not show a substantial improvement in the proportion of produced renewable energy over total produced energy after having ratified the Kyoto Protocol first and the Doha Agreement successively. Countries joining the EU-ETS (such as United Kingdom and Denmark) are characterized by a greater growth compared to the others. Notice also the interesting behaviors of New Zealand and Hungary. The coloured band refers to the interval given by [μ+σ, μ-σ]"))
def set_title_1(event):
    if event.new in euts:
        column_prod[0]=f'## Renweable Energy Share: {event.new} (In EU-ETS)'
    else:
        column_prod[0]=f'## Renweable Energy Share: {event.new} (Not in EU-ETS)'

country.param.watch(set_title_1, 'value')

# Set y ticks format
def formatter(value):
    return str(value) + ' %'

# Add context Official Start Kyoto Protocol and Doha Agreement
context = hv.Text('2005', 70, 'Kyoto Protocol official start', 8).opts(text_font_style = 'italic')
context2 = hv.Text('2012', 90, 'Doha Agreement official start', 8).opts(text_font_style = 'italic')

vline = hv.VLine(2005).opts(alpha = 1, color = PALETTE_COLOR_BLINDNESS[4])
vline2 = hv.VLine(2012).opts(alpha = 0.1, color = PALETTE_COLOR_BLINDNESS[5])

share_plot = (main_plot.opts(tools=[hover], 
                            toolbar = 'above', 
                            ) * 
              kyotoI_plot *
              kyotoII_plot *
              share_band *
              vline *
              vline2 *
              context * 
              context2
              )

time_series_figure_share = pn.Column(column_prod, #select_mode_kyotoI, select_mode_kyotoII, #country,
                                     share_plot.opts(legend_position='right', padding = (0.4, 0.4),
                                                     xlim=(1990, 2021), ylim = (0,110), 
                                                     ylabel = 'Renewable Production/Total Production',
                                                     xticks=5, yticks=5,
                                                     yformatter = formatter,
                                                     text_font_style = 'bold italic'
                                                     ).panel(width=700, height = 400))

# time_series_figure_share.show()
# # time_series_figure_share.servable()


## Time serie Production

### Combine mean plot and main plot

In [10]:
## Combine Plot and Pipeline
hover = HoverTool(tooltips=[("Year", "@{Year}"), 
                            ("Value", "@{Renewable electricity productions (TWh)} Twh"),
                            ("Relative Change Production", "@{PercentageChangeProduction}%")
                           ],
                  mode = 'vline')

main_plot = ipipeline1.hvplot(x = 'Year',
                              y = 'Renewable electricity productions (TWh)', 
                              size = 'PercentageChangeProduction',
                              width=200, height=400,
                              alpha = 1, line_color= select_mode_main, 
                              label = 'Renewable Production (Twh)', 
                              )


kyotoI_plot = ipipeline1.hvplot(x = 'Kyoto I', 
                                y = 'Renewable electricity productions (TWh) Kyoto',
                                width=200, height=400,
                                alpha = 1, line_color= select_mode_kyotoI,
                                label = 'Kyoto Protocol first part'
                                ).opts(tools=[""], toolbar=None)


kyotoII_plot = ipipeline1.hvplot(x = 'Kyoto II', 
                                 y = 'Renewable electricity productions (TWh) Kyoto', 
                                 width=200, height=400, 
                                 label = 'Kyoto Protocol second part',
                                 alpha = 1, line_color= select_mode_kyotoII, 
                                 ).opts(tools=[""], toolbar=None)
#                                        )

## Production mean band
mean_prod = idc.groupby('Year').mean()[['Renewable electricity productions (TWh)', 'Eco-friendly Production/Total Production Share(%)']]
mean_prod['Upper'] = mean_prod['Renewable electricity productions (TWh)'] + idc.groupby('Year').std()['Renewable electricity productions (TWh)']
mean_prod['Lower'] = mean_prod['Renewable electricity productions (TWh)'] - idc.groupby('Year').std()['Renewable electricity productions (TWh)']


mean_p = mean_prod.hvplot(x = 'Year',
                          y = 'Renewable electricity productions (TWh)',
                          kind = 'line',
                          label = 'Mean Production',
                          alpha = 1, 
                          line_color= select_mode_mean.value,
                          ylim = (0,110),
                          xlim= (1990,2020)
                          )

band_p = mean_prod.hvplot(x='Year', y='Upper', y2='Lower',
                          kind = 'area',
                          color = select_mode_mean.value,
                          alpha = 0.1
                         )
prod_band = (mean_p * band_p.opts(tools = [], toolbar = None))

## Define interactive title
column_prod_share = pn.Column(pn.Row('## Renweable Energy Production: United Kingdom (In EU-ETS)'),
                             pn.Row('The renewable production slowly grow up for most of the countries. Germany and Japan have the fastest development respectively since Kyoto Protocol official start and Doha start. The coloured band refers to the interval given by [μ-σ;μ+σ]')) 
def set_title_2(event):
    if event.new in euts:
        column_prod_share[0]=f'## Renweable Energy Production: {event.new} (In EU-ETS)'
    else:
        column_prod_share[0]=f'## Renweable Energy Production: {event.new} (Not in EU-ETS)'

country1.param.watch(set_title_2, 'value')

## Define axis y style 
def formatter(value):
    return str(value) + ' TWh'

## Write context lines (Kyoto Protocol starts, Doha starts)
context = hv.Text('2005', 220, 'Kyoto Protocol official start', 8).opts(text_font_style = 'italic')
context2 = hv.Text('2012', 240, 'Doha Agreement official start', 8).opts(text_font_style = 'italic')

vline = hv.VLine(2005).opts(alpha = 1, color = PALETTE[4])
vline2 = hv.VLine(2012).opts(alpha = 0.1, color = PALETTE[5])


## Combine plots
plot = (main_plot.opts(tools = [hover], 
                      toolbar = 'above') *
        kyotoI_plot * kyotoII_plot *
        prod_band *
        vline *
        vline2 *
        context *
        context2 
        )

## Define a column for plot, widgets and titles
time_series_figure = pn.Column(column_prod_share,
#                                select_mode_mean,
                               select_mode_kyotoI, select_mode_kyotoII,
                               plot.opts(legend_position='right',
                                         padding = (0.4, 0.4),
                                         xlim=(1990, 2021), ylim = (0,300), 
                                         ylabel = 'Renewable Production',
                                         xticks=5, yticks=5,
                                         yformatter = formatter,
                                         text_font_style = 'bold italic'
                                         ).panel(width=700, height = 400, margin = (20,0))
                                )
# time_series_figure.show()
# time_series_figure.servable()



## Dashboard

In [11]:
## Import Lollipop
fname = "Lollipop.html"
# html_file = open(PATH_DATA/fname, 'r', encoding='utf-8')
html_file = open(fname, 'r', encoding='utf-8')
source_code = html_file.read() 
lollipop = pn.pane.HTML(source_code)
lollipop

HTML(str, sizing_mode='stretch_width')

In [12]:
image = pn.pane.PNG('KyotoProtocol.png',
                                 width = 300, height = 300, align = 'center'

                            )
test = pn.pane.Markdown('The Kyoto Protocol is an international treaty signed and ratified by 192 countries all over the world. It is considered the first and most important agreement to reduce greenhouse gas emissions that are linked to global warming. The Kyoto Protocol was adopted on 11 December 1997 but officially entered into force on 16 February 2005, when 55 Nations, representing at least 55% of global gas emissions, ratified the treaty. Its first part ended in 2012. Yet, Kyoto protocol was successively extended to 2020, thanks to Doha Agreement. The aim was to make the percentage of gas emissions of each signing country drop compared to 1990’s levels, and the results had to be achieved at any level (i.e. energetic industry, transportation, manufacturing industry, etc…). Thus, the Kyoto Protocol represents the first way to change the world habits to a more sustainable life, where energy is produced thanks to low-carbon emissions materials. For the energy level the EU-ETS^1 plays an important role for the decrease and monitor of gas emissions. Here, we will consider the 38 Countries with highest HDI^2 () (i.e. HDI of 0.8 and above) which also signed the Kyoto Protocol.'
                        ,width=100)
cap1 = pn.pane.Markdown("1 EU-ETS (European Union Emissions Trading System) = it is a ‘cap and trade’ scheme where a limit is placed to the right to emit specified pollutants. It covers around 45% of EU’s greenhouse gas emissions."
                                     )
cap2 = pn.pane.Markdown("2 HDI (Human Development Index) = is a summary measure of average achievement in key dimensions of human development: a long and healthy life, being knowledgeable and have a decent standard of living. The HDI is the geometric mean of normalized indices for each of the three dimensions. It ranges 0-1.")


In [14]:
pn.extension(sizing_mode = 'stretch_width')
# Add all plot together into an interactive dashboard
template = pn.template.MaterialTemplate(
            title = 'Did Kyoto Protocol have effects on renewable production over the years?',
#             sidebar=[pn.pane.PNG(PATH_DATA/'KyotoProtocol.png',
            main=[pn.Row('## Please select color vision'), 
                  pn.Row(lollipop),
#                   lollipop,
                  pn.Spacer(height = 700),
                  pn.Row('# Please First of all select color vision',select_mode_main,
                         '## Select Country', country, country1),
                  pn.Row(time_series_figure_share, 
                         time_series_figure),
                  pn.Row(itable.panel(width = 500)
                       )
                  ]
)
template.sidebar.append(image)
template.sidebar.append(test)
template.sidebar.append(cap1)
template.sidebar.append(cap2)
template.show()
# template.servable()
# pn.serve(template, threaded=True, title='Interactive dashboard', 
#         ip_address = '0.0.0.0', port = 8000, allow_websocket_origin = app_name.azurewebsites.net)


Launching server at http://localhost:51285
